In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

In [2]:
# parameters
customer_id = '134'                                                                           
formatted_attribute = 'material'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_') #use this to create column names where spaces are _

buckets = """V-Belt"""

regex_pattern = r"(?i)(\b\d*\.\d* In\w* Top Width)|(\b\d* In\w* Top Width)|(\b\d*\/\d* In\w* Top Width)|(\b\d*\-\d*\/\d* in Top Width)"

value='%n/a%'
customer_name='%motionindustriesus%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}' #use this to create columns

In [3]:
# metric = ' mm'

# range_string = """0 - 2000 mm
# 2001 - 4000 mm
# 4001 - 6000 mm
# 6001 - 8000 mm
# 10001 - 12000 mm
# 12001 - 14000 mm
# Greater than 14000 mm"""

In [4]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)

In [5]:
df = query_from_file(file_name='../query/top_width.sql', params=params) #create query
df[df['value']!='n/a']

attribute     value      customers_name external_id  \
1      material  Neoprene  motionindustriesus    07104872   
2      material  Neoprene  motionindustriesus    04687424   
3      material  Neoprene  motionindustriesus    04718432   
4      material  Neoprene  motionindustriesus    07104743   
5      material  Neoprene  motionindustriesus    07104675   
...         ...       ...                 ...         ...   
74315  material     Metal  motionindustriesus    07367926   
74323  material   Plastic  motionindustriesus    05686038   
74340  material     Metal  motionindustriesus    04181398   
74350  material     Metal  motionindustriesus    03254031   
74357  material     Metal  motionindustriesus    05020307   

                                                                                                                         long_desc  \
1       ZR Synchronous Timing Belt - XXH, 1.2500 in Pitch, 90.0000 in Overall Length, 4.0000 in Width, 72 Teeth, Neoprene Material   
2       ZR Synchronous Timing Belt - XXH, 1.2500 in Pitch, 70.0000 in Overall Length, 3.0000 in Width, 56 Teeth, Neoprene Material   
3      ZR Synchronous Timing Belt - XXH, 1.2500 in Pitch, 120.0000 in Overall Length, 5.0000 in Width, 96 Teeth, Neoprene Material   
4      ZR Synchronous Timing Belt - XXH, 1.2500 in Pitch, 120.0000 in Overall Length, 4.0000 in Width, 96 Teeth, Neoprene Material   
5      ZR Synchronous Timing Belt - XXH, 1.2500 in Pitch, 100.0000 in Overall Length, 5.0000 in Width, 80 Teeth, Neoprene Material   
...                                                                                                                            ...   
74315                                                                                      01177591 48\" METAL DUST MOP FRAME(MSC)   
74323                                                                                       0108738020 32MM PLASTIC CREVICE NOZZLE   
74340                                                                                                  00547216 HANDLE METAL BROOM   
74350                                                                                         00301 LIBMAN CORN BROOM W/METAL HNDL   
74357                                                                                             0000 FINE STEEL WOOL 16PADS/PACK   

      resolution  
1       standard  
2       standard  
3       standard  
4       standard  
5       standard  
...          ...  
74315   standard  
74323   standard  
74340   standard  
74350   standard  
74357   standard  

[44972 rows x 6 columns]

In [6]:
# Find a list of values to iterate
df_strategy = query_from_file(file_name='../asset/easy.sql', params=params) #create query
df_strategy
list_values=df_strategy['value'].to_list()
list_values

['Aluminum',
 'Aluminum Oxide',
 'Cast Iron',
 'Chloroprene',
 'Cotton',
 'Denim',
 'EPDM',
 'Fabric',
 'Flannel',
 'Fleece',
 'Foam',
 'HDPE',
 'Horsehair',
 'Iron',
 'Kevlar',
 'Latex',
 'Mesh',
 'Metal',
 'Microfiber',
 'Neoprene',
 'Nylon',
 'Palmyra',
 'Paper',
 'Plastic',
 'Polyester',
 'Polyethylene',
 'Polypropylene',
 'Polystyrene',
 'Polyurethane',
 'Polyvinyl',
 'Resin',
 'Rubber',
 'Silica',
 'Stainless Steel',
 'Steel',
 'Synthetic',
 'Tampico',
 'Terry',
 'Wood',
 'Zinc Alloy']

In [7]:
kwp=KeywordProcessor()
kwp.add_keywords_from_list(list_values)
df['long_desc'].apply(lambda x: kwp.extract_keywords(x)) #

In [10]:
df.shape[0]*2

148716

In [11]:
#Find non-blank matches
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup) # get all the matches that are not blank
    return list(set(tmp_matches))

In [12]:
df['matches'] = df['long_desc'].apply(lambda x: kwp.extract_keywords(x))
df['matches'].explode().value_counts()

Cast Iron          13392
Neoprene            4785
Polyurethane        4015
Rubber              2726
Chloroprene         1300
Cotton               932
Steel                862
EPDM                 685
Synthetic            679
Plastic              520
Wood                 393
Polypropylene        315
Polyester            272
Polyethylene         230
Microfiber           190
Metal                166
Fabric               164
Tampico              162
Nylon                157
Paper                130
Palmyra              113
Horsehair             92
Aluminum              76
HDPE                  65
Foam                  61
Zinc Alloy            28
Polystyrene           27
Stainless Steel       27
Iron                  25
Terry                 16
Latex                 16
Denim                  9
Resin                  8
Mesh                   6
Flannel                6
Fleece                 5
Polyvinyl              4
Name: matches, dtype: int64

In [13]:
# df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))

#This is only for material
df['matches']=df['value']
df

attribute     value      customers_name external_id  \
0      material       n/a  motionindustriesus    06860760   
1      material  Neoprene  motionindustriesus    07104872   
2      material  Neoprene  motionindustriesus    04687424   
3      material  Neoprene  motionindustriesus    04718432   
4      material  Neoprene  motionindustriesus    07104743   
...         ...       ...                 ...         ...   
74353  material       n/a  motionindustriesus    05215061   
74354  material       n/a  motionindustriesus    03256314   
74355  material       n/a  motionindustriesus    03256313   
74356  material       n/a  motionindustriesus    03256303   
74357  material     Metal  motionindustriesus    05020307   

                                                                                                                         long_desc  \
0                                              HYGEN™ Disposable Microfiber - White Color, 12 in Wide, 12 in Long, 4 Per Container   
1       ZR Synchronous Timing Belt - XXH, 1.2500 in Pitch, 90.0000 in Overall Length, 4.0000 in Width, 72 Teeth, Neoprene Material   
2       ZR Synchronous Timing Belt - XXH, 1.2500 in Pitch, 70.0000 in Overall Length, 3.0000 in Width, 56 Teeth, Neoprene Material   
3      ZR Synchronous Timing Belt - XXH, 1.2500 in Pitch, 120.0000 in Overall Length, 5.0000 in Width, 96 Teeth, Neoprene Material   
4      ZR Synchronous Timing Belt - XXH, 1.2500 in Pitch, 120.0000 in Overall Length, 4.0000 in Width, 96 Teeth, Neoprene Material   
...                                                                                                                            ...   
74353                                                                                            000524 DUST HEAD 5\" X 24\" 12/CA   
74354                                                                                               00028 #28 STICK MOP (20 OUNCE)   
74355                                                                                                   0002 24 OZ COTTON MOP HEAD   
74356                                                                                                   0001 32 OZ COTTON MOP HEAD   
74357                                                                                             0000 FINE STEEL WOOL 16PADS/PACK   

      resolution   matches  
0       standard       n/a  
1       standard  Neoprene  
2       standard  Neoprene  
3       standard  Neoprene  
4       standard  Neoprene  
...          ...       ...  
74353   standard       n/a  
74354   standard       n/a  
74355   standard       n/a  
74356   standard       n/a  
74357   standard     Metal  

[74358 rows x 7 columns]

In [14]:
def na_to_gsheet(df):
    url = 'https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0'
    wks = 'n/a'
    non_matches = df[df['matches'].astype(str) == '[]']
    non_matches[curation_col] = r'n/a'
    gc = gcloud.Gsheets()
    gc.save_to_gsheet(url, wks, non_matches)

In [17]:
# na_to_gsheet(df)

In [ ]:
matches = df[df['matches'].astype(str) != '[]'] #Find all non blank matches
matches

#### matches['matches'].astype(str).value_counts()[:25] #Return index of matches and how often they occur

#matches['matches'] = matches['matches'].apply(lambda x: float(literal_eval(x)))

In [ ]:
# list(set(matches['matches'].iloc[0])) #do this for everyone

In [ ]:
# matches[curation_col]=matches['matches'].apply(lambda quniue: list(set(quniue))) 

In [ ]:
matches['external_id']=matches['external_id'].apply('="{}"'.format) 
matches[curation_col] = matches['matches'] 

In [ ]:
matches

In [16]:
# gc = gcloud.Gsheets()

FileNotFoundError: [Errno 2] No such file or directory: 'client_secret.json'

In [ ]:
# gc.save_to_gsheet(url,'ul',matches)

# send to the folder for upload

In [ ]:
def looks_good(customer, attribute, df, matches):
    
#     G:\Shared drives\GroupBy Public\Customer Success\.Enrich\Platform Upload Trail
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload'
    non_matches = df[df['matches'].astype(str) == '[]']
    non_matches[curation_col] = r'n/a'
    non_matches.to_csv(f'{drive_path}/Motion - {attribute} - na upload_Shawn.csv',index=False)
    matches.to_csv(f'{drive_path}/Motion - {attribute} - match upload_Shawn.csv',index=False)

In [ ]:
looks_good('Motion Industries', attribute, df, matches)